In [ ]:
from Bio import SeqIO
import os
import subprocess

#paths 
reference_genome = 'data/NC_045512.2.fasta'
genome_file = 'data/selected_1000_genomes.fasta'
genome_dir = 'data/selected_1000_genomes/'
output_dir = 'results/alignment_output/'
gatekeeper_path = 'tools/GATEkeeper/bin/GATEkeeper'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to extract individual genomes from the multi-genome FASTA file
def extract_individual_genomes(multi_genome_file, output_dir):
    genome_files = []
    for record in SeqIO.parse(multi_genome_file, "fasta"):
        genome_name = record.id
        genome_file = os.path.join(output_dir, f"{genome_name}.fasta")
        
        # Write individual genome sequence to a file
        with open(genome_file, "w") as f:
            SeqIO.write(record, f, "fasta")
        
        genome_files.append(genome_file)
    return genome_files

# Run GATEkeeper for pairwise genome alignment
def run_gatekeeper_vcf(reference_genome, genome_file, output_dir):
    # Generate output VCF file name
    vcf_output = os.path.join(output_dir, f"{os.path.basename(genome_file)}_vs_{os.path.basename(reference_genome)}.vcf")
    
    # Construct the command to run GATEkeeper
    command = [
        gatekeeper_path,
        "-r", reference_genome,
        "-q", genome_file,
        "-o", vcf_output
    ]
    
    # Run the command
    subprocess.run(command, check=True)
    return vcf_output

# Main execution
if __name__ == "__main__":
    # If you have a multi-genome file (split it into individual genomes)
    if genome_file.endswith('.fasta'):
        print("Extracting individual genomes from the multi-genome file...")
        genome_files = extract_individual_genomes(genome_file, output_dir)
    else:
        # If you already have individual genome files
        genome_files = [os.path.join(genome_dir, f) for f in os.listdir(genome_dir) if f.endswith('.fasta')]
    
    # Align each genome against the reference genome
    for genome_file in genome_files:
        print(f"Aligning genome: {os.path.basename(genome_file)}...")
        vcf_file = run_gatekeeper_vcf(reference_genome, genome_file, output_dir)
        print(f"VCF output for {os.path.basename(genome_file)} stored in: {vcf_file}")
    
    print("Alignment completed for all genomes.")


In [ ]:
import vcfpy
import os

def merge_vcf_files(vcf_files, output_vcf_file):
    # Create a VCF reader for the first file to extract the header
    with vcfpy.Reader.from_path(vcf_files[0]) as reader:
        # Create a VCF writer with the header from the first VCF file
        with vcfpy.Writer.from_path(output_vcf_file, reader.header) as writer:
            
            # Iterate over each VCF file and add its records to the merged output file
            for vcf_file in vcf_files:
                print(f"Processing file: {vcf_file}")
                with vcfpy.Reader.from_path(vcf_file) as reader:
                    # Write each record from the current VCF file to the output file
                    for record in reader:
                        writer.write_record(record)
    
    print(f"Merged VCF file saved as: {output_vcf_file}")


import os

# Define a generic output directory
output_dir = 'results/alignment_output/'

# Collect the VCF files from the output directory
vcf_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.vcf')]

# Specify the path to your merged output VCF file
output_vcf_file = os.path.join(output_dir, 'merged_genomes_vs_NC_045512.2.vcf')

# Call the merge function
merge_vcf_files(vcf_files, output_vcf_file)


In [ ]:
import warnings
import vcfpy

# Suppress specific warning related to unknown filters in VCF
warnings.filterwarnings("ignore", category=UserWarning, message="UnknownFilter: Filter not found in header")

# Path to the merged VCF file (adjust this to your specific file path)
vcf_file = 'merged_genomes_vs_NC_045512.2.vcf'

# Read and process the VCF file
with vcfpy.Reader.from_path(vcf_file) as reader:
    # Process each record (variant)
    for record in reader:
        # Example: Accessing some basic data for mutation analysis
        chrom = record.CHROM
        pos = record.POS
        ref = record.REF
        alt = record.ALT
        info = record.INFO
        # You can add your logic here to analyze the SNPs and Indels
        
        print(f"Chromosome: {chrom}, Position: {pos}, REF: {ref}, ALT: {alt}, Info: {info}")
        
        # Continue processing records to extract mutations, count SNPs/Indels, etc.



In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.discrete.discrete_model import NegativeBinomial
import numpy as np

# Paths
vcf_dir = '/Downloads' # Directory containing VCF files
reference_genome = "NC_045512.2"  # Reference genome ID

# Function to parse VCF files
def parse_vcf(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            chrom = parts[0]
            pos = int(parts[1])
            ref = parts[3]
            alt = parts[4]
            info = parts[7]
            # Identify SNP or Indel
            mutation_type = "SNP" if len(ref) == 1 and len(alt) == 1 else "Indel"
            data.append({"chrom": chrom, "pos": pos, "ref": ref, "alt": alt, "type": mutation_type})
    return pd.DataFrame(data)

# Combine mutations from all VCF files
all_mutations = []
for vcf_file in os.listdir(vcf_dir):
    if vcf_file.endswith(".vcf"):
        vcf_path = os.path.join(vcf_dir, vcf_file)
        mutations = parse_vcf(vcf_path)
        all_mutations.append(mutations)
combined_df = pd.concat(all_mutations)

# Count mutation types
mutation_counts = combined_df['type'].value_counts()

# Plot 1: Bar Chart for SNP and Indel counts
plt.figure(figsize=(8, 5))
sns.barplot(x=mutation_counts.index, y=mutation_counts.values)
plt.title("Counts of Mutation Types (SNPs vs Indels)")
plt.xlabel("Mutation Type")
plt.ylabel("Count")
plt.show()

# Plot 2: Manhattan Plot for Mutation Frequency
mutation_frequency = combined_df.groupby("pos").size()
plt.figure(figsize=(12, 6))
plt.scatter(mutation_frequency.index, mutation_frequency.values, alpha=0.5, c="blue", s=10)
plt.title("Manhattan Plot: Mutation Frequency (NC_045512)")
plt.xlabel("Position on Genome in bp")
plt.ylabel("Frequency")
plt.show()

# Plot 3: Histogram of SNP Frequencies
snp_positions = combined_df[combined_df["type"] == "SNP"]["pos"]
plt.figure(figsize=(10, 6))
plt.hist(snp_positions, bins=50, color="green", alpha=0.7)
plt.title("Histogram of SNP Frequencies")
plt.xlabel("Position on Genome")
plt.ylabel("Count")
plt.show()

# Statistical Analysis: Negative Binomial Model
# Step 1: Prepare data for the model
mutation_frequency_df = mutation_frequency.reset_index(name="frequency")
mutation_frequency_df.columns = ["pos", "frequency"]

# Fit Negative Binomial Model (adjusting the code for correct usage)
X = np.ones((len(mutation_frequency_df), 1))  # Dummy predictor (intercept only)
y = mutation_frequency_df["frequency"]
model = NegativeBinomial(y, X)
result = model.fit()

# Extract p-values for significant mutations (p-value < 0.0001)
mutation_frequency_df["p_value"] = result.pvalues
significant_mutations = mutation_frequency_df[mutation_frequency_df["p_value"] < 0.0001]



In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.discrete.discrete_model import NegativeBinomial
import numpy as np

vcf_file ='merged_genomes_vs_NC_045512.2.vcf'
plt.figure(figsize=(12, 6))
plt.scatter(mutation_frequency.index, mutation_frequency.values, alpha=0.5, c="blue", s=10)
plt.title("Manhattan Plot: Mutation Frequency (NC_045512)")
plt.xlabel("Position on Genome in bp")
plt.ylabel("Frequency")
plt.show()
